Carregando os dados necessários

In [15]:
from pandas import DataFrame, read_csv, to_datetime
from os import path

datadir = 'C:/Users/User/Desktop/data_science/mentoria_data_mundo/projeto_ecommerce/_data/clean'

df_clientes = read_csv(path.join(datadir, 'e_commerce_clientes.csv'))
df_carrinhos = read_csv(path.join(datadir, 'e_commerce_carrinhos.csv'))
df_pedidos = read_csv(path.join(datadir, 'e_commerce_pedidos.csv'))

,Unnamed: 0,id,data,data_pagamento,data_cancelamento,numero_pedido,status,pagamento,produto,sku,...,entrega_cidade,entrega_estado,entrega_cep,utm_source,utm_campaign,utm_medium,utm_content,utm_term,customizacao,id_cliente
0,1,117569438,04/08/2024 18:26:40,04/08/2024 18:27,NaN,1042552530523300,Pagamento aprovado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Brasilia,Distrito Federal,72455490,NaN,NaN,NaN,NaN,NaN,NaN,31d69aed10
1,2,117385907,01/08/2024 19:08:02,01/08/2024 19:11,NaN,1042552893870166,Pagamento aprovado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5
2,3,117385794,01/08/2024 19:05:53,NaN,01/08/2024 19:12,1042552190094688,Cancelado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5
3,4,117382873,01/08/2024 18:12:14,NaN,01/08/2024 19:13,1042552273445602,Cancelado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5
4,5,117382771,01/08/2024 18:09:41,NaN,01/08/2024 19:13,1042552782939597,Cancelado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,141,112676891,22/05/2024 13:22:56,22/05/2024 13:22,22/05/2024 13:27,1042552727006222,Cancelado,Cartão de crédito,Teste,4P8E3PJHB,...,Florianopolis,Santa Catarina,88058000,NaN,NaN,NaN,NaN,NaN,NaN,0db3cf4f27
141,142,112676767,22/05/2024 13:21:14,NaN,22/05/2024 15:01,1042552431219898,Cancelado,Pix,Teste,4P8E3PJHB,...,Florianopolis,Santa Catarina,88058000,NaN,NaN,NaN,NaN,NaN,NaN,0db3cf4f27
142,143,112675088,22/05/2024 12:54:55,NaN,22/05/2024 14:45,1042552609058447,Cancelado,Pix,BRINCO ARGOLA SOL E LUA,JRM3YWN95,...,Florianopolis,Santa Catarina,88058000,NaN,NaN,NaN,NaN,NaN,NaN,0db3cf4f27
143,144,112670127,22/05/2024 11:37:57,22/05/2024 11:37,22/05/2024 11:47,1042552577521467,Cancelado,Cartão de crédito,Teste,JMPBRU537,...,Florianopolis,Santa Catarina,88058000,NaN,NaN,NaN,NaN,NaN,NaN,0db3cf4f27


Realizando os tratamentos dos dados necessários

In [32]:
# Do dataframe 'df_pedidos' Iremos pegar a coluna 'total_pago' e a coluna 'id_cliente', assim teremos o valor total pago por cliente, além da data.
# A ideia é dividir em dois grupos, o ticket médio dos clientes em datas normais, e em datas especiais.

# Trocando as ',' por '.' para converter para float na coluna 'total_pago'
df_pedidos['total_pago'] = (df_pedidos['total_pago']
 .replace(',', '.', regex=True)
 .astype({'total_pago': float}))

# Trocando o tipo da coluna 'data' para datetime (formato do pandas para datas)
df_pedidos['data'] = to_datetime(df_pedidos['data'], format='%d/%m/%Y %H:%M:%S')

# Filtrando apenas as colunas necessárias do dataframe 'df_pedidos'
df_pedidos = df_pedidos.filter(['id_cliente', 'total_pago', 'data'])

# Separando as compras feitas em datas especiais e datas normais
## Vamos criar a lista com as datas especiais
datas_especiais = to_datetime([
    '2024/01/01', '2024/01/06', '2024/01/25', '2024/02/10', '2024/02/11', 
    '2024/02/12', '2024/02/13', '2024/02/14', '2024/02/24', '2024/03/08', 
    '2024/03/10', '2024/03/19', '2024/03/29', '2024/04/01', '2024/04/21', 
    '2024/04/22', '2024/04/25', '2024/04/30', '2024/05/01', '2024/05/13', 
    '2024/05/31', '2024/06/05', '2024/06/12', '2024/06/24', '2024/07/02', 
    '2024/07/09', '2024/07/20', '2024/08/11', '2024/08/15', '2024/08/19', 
    '2024/09/07', '2024/09/29', '2024/10/12', '2024/10/15', '2024/10/28', 
    '2024/11/02', '2024/11/15', '2024/11/20', '2024/12/24', '2024/12/25', '2024/12/31'], 
    format='%Y/%m/%d')

## Agora criaremos outro dataframe apenas com as datas_especiais


Timestamp('2024-08-04 18:26:40')